For this kata, we're given an image in which some object of interest (e.g. a face, or a license plate, or an aircraft) appears as a large block of contiguous pixels all of the same colour. (Probably some image-processing has already occurred to achieve this, but we needn't worry about that.) We want to find the centre of the object in the image.

We'll do this by finding which pixels of the given colour have maximum depth. The depth of a pixel P is the minimum number of steps (up, down, left, or right) you have to take from P to reach either a pixel of a different colour or the edge of the image.

In the picture, the red pixel marked "3" has a depth of 3: it takes at least 3 steps from there to reach something other than another red pixel. Note that the steps need not be all in the same direction. Only one red pixel has depth 3: the one right in the middle of the red region. Similarly, the blue pixel marked "2" has a depth of 2 (but it is not the only one with this depth). The green and purple pixels all have depth 1.

The pixels of a given colour with the largest depth will be found at the centre of the biggest solid region(s) of that colour. Those are the ones we want.

The function you'll write (central_pixels) belongs to the following data structure:

class Image:
  def __init__(self, data, w, h): 
    self.pixels = data
    self.width = w
    self.height = h

The image data consists of a one-dimensional array pixels of unsigned integers (or just integers, in languages that don't have unsigned integers as such), which correspond to pixels in row-by-row order. (That is, the top row of pixels comes first, from left to right, then the second row, and so on, with the pixel in the bottom right corner last of all.) The values of the pixels array elements represent colours via some one-to-one mapping whose details need not concern us.

The central_pixels function should find and return all the positions (pixels array indices) of the pixels having the greatest depth among all pixels of colour colour).

Note 1. The final test in the suite (Big_Test) is a 16-megapixel image (1 megapixel in the Python version), so you will need to consider the time and space requirements of your solution for images up to that size.

Note 2. The order of pixel positions in the returned array is not important; sort them however you like.

Hint. It is possible to get this done in two passes through the pixel data.

In [160]:
class Image:
  def __init__(self, data, w, h): 
    self.pixels = data
    self.width = w
    self.height = h

In [248]:
class Central_Pixels_Finder(Image):

    def central_pixels(self, colour):
        # class variables
        w = self.width
        h = self.height
        data = self.pixels

        # initialise list of depths with 0 for all positions
        depths = [0 for i in range(len(data))]

        # cycle through data forwards looking for instances of required colour
        for i in range(len(data)):
            if data[i] == colour:
                # if in first row or column, set depth to 1
                if i < w or i % w == 0:
                    depths[i] = 1
                # otherwise compare it with the previous entry in the same row, and the
                # previous entry in the same column
                else:
                    depths[i] = 1 + min( depths[i-1], depths[i-w] )
      
        # now cycle through data backwards looking for instances of required colour
        positions = []
        max_depth = 1
        for i in range(len(data)-1, -1, -1):
            if data[i] == colour:
                # if in final row or column, set depth to 1
                if i >= len(data) - w or (i + 1) % w == 0:
                    depths[i] = 1
                # otherwise compare it with the following entry in the same row, and the 
                # following entry in the same column
                else:
                    depths[i] = min( depths[i], 1 + depths[i+1], 1 + depths[i+w] )
              
                # if this entry has the maximum depth, add to the results list
                if depths[i] == max_depth:
                    positions.append(i)
                # if it exceeds the maximum depth, reset the max depth and results list
                elif depths[i] > max_depth:
                    max_depth = depths[i]
                    positions = [i]

        return sorted(positions)

In [249]:
image = Central_Pixels_Finder( [1,1,4,4,4,4,2,2,2,2,
                                1,1,1,1,2,2,2,2,2,2,
                                1,1,1,1,2,2,2,2,2,2,
                                1,1,1,1,1,3,2,2,2,2,
                                1,1,1,1,1,3,3,3,2,2,
                                1,1,1,1,1,1,3,3,3,3], 10, 6 )
print(image.central_pixels(1))
print(image.central_pixels(2))
print(image.central_pixels(3))
print(image.central_pixels(4))
print(image.central_pixels(5))

[32]
[16, 17, 18, 26, 27, 28, 38]
[35, 45, 46, 47, 56, 57, 58, 59]
[2, 3, 4, 5]
[]


This solution is too inefficient to pass CodeWars test cases

It also doesn't take account of diagonals

In [239]:
import numpy as np
class Central_Pixels_Finder(Image):

  def process_rows(self, data, colour):
    block_reset = []
    colours = []
    i = count = 0

    # for each row, populate array with count of colour
    for row in data:
      block_reset.append(i)
      count = 0
      for col in row:
        if col == colour:
          count += 1
          colours.append(count)
        else:
          colours.append(0)
          if count != 0:
            block_reset.append(i)
            count = 0
        i += 1
    block_reset.append(-1)

    # for each block of colour, find the max, and replace each cell with the minimum of the count, and (max+1-count)
    for i in range(1,len(block_reset)):
      if block_reset[i] == -1:
        block = colours[block_reset[i-1]:]
      else:
        block = colours[block_reset[i-1]:block_reset[i]]
      block_max = max(block)
      block_adj = [min(i, block_max+1-i) if i != 0 else 0 for i in block]
      if i == 1:
        part1 = []
      else:
        part1 = colours[0:block_reset[i-1]]
      part2 = block_adj
      if block_reset[i] == -1:
        part3 = []
      else:
        part3 = colours[block_reset[i]:]
      colours = part1 + part2 + part3
    return colours

  def central_pixels(self, colour):
    w = self.width
    h = self.height
    data = np.array(self.pixels)

    # arrange the data into rows and columns, and process
    rows = self.process_rows(data.reshape(h,w), colour)

    # now rearrange the data so the columns become rows, and process
    cols = np.array(self.process_rows(data.reshape(h,w).T, colour))
    cols = cols.reshape(w,h)

    # rearrange back so the rows become rows again
    cols = cols.reshape(w,h).T
    cols = cols.reshape(1, w*h).tolist()[0]

    # now compare the two lists and for each entry take the minimum
    counts = [min(rows[i], cols[i]) for i in range(len(rows))]

    # find all instances of the max entry
    max_count = max(counts)
    return [i for i in range(len(counts)) if counts[i] == max_count and max_count > 0]